# Intro to Using the NetHack Learning Dataset

There are two different sets of trajectories included in the NetHack Learning Dataset:
- **NLD-NAO**: state-only trajectories from 1.5 M human games played on nethack.alt.org
- **NLD-AA**: state-action-score trajectories from 100k NLE games played by the symbolic-bot winner of the 2021 NetHack Challenge

We also supply a small "taster" dataset, for quick iteration and playing around:
- **NLD-AA-Taster**: ~2,000 randomly chosen trajectories from **NLD-AA**

These trajectories can be used with the `TtyrecDataset` tool which allows for efficiently training on the datasets.  This tutorial describes how to create and use and visualize the dataset, using **NLD-AA-Taster**.



## Downloading the Data

For the time being data is available through various WeTransfer links in the DATASET.md file. Although generally this requires a browser to interface to download, it is also possible to use the command line (see here).

In this case, we use a publically available unzipped version of **NLD-AA-Taster** available on GitHub (or you can access the [zipped release]()).


## Install NLE

Make sure you have `nle` installed by following the instructions [in the repo README here](https://github.com/facebookresearch/nle). Either clone and install, or use pip. In this case, Colab struggles a bit to find cmake so we build from source:

## Setting up the Database

Adding datasets is easy - all you need is the path to the unzipped directory.

**NOTE** We call different functions to add trajectories generated by NLE (such as **NLD-AA**, **NLD-AA-Taster** or your own dataset) versus those generated from NAO (**NLD-NAO**).  

In [14]:
import nle.dataset as nld

In [15]:
# 1. Get the paths for your unzipped datasets
path_to_nld_aa_taster = "./data/nld-aa-taster/nle_data"

# 2. Chose a database name/path. By default, most methods with use nld.db.DB (='ttyrecs.db')
dbfilename = "ttyrecs.db"

if not nld.db.exists(dbfilename):
    # 3. Create the db and add the directory
    nld.db.create(dbfilename)
    nld.add_nledata_directory(path_to_nld_aa_taster, "taster-dataset", dbfilename)


# NB: To add the NLE-AA data, or any data generated from nle, use `add_nledata_directory`.
# nld.add_nledata_directory(path_to_nld_aa, "nld-aa", dbfilename)

# NB: To add the NLE-NAO data, use the `add_altorg_directory`.
# nld.add_altorg_directory(path_to_nld_nao, "nld-nao", dbfilename)


You can inspect the dataset using the database tooling:

In [16]:
# Create a connection to specify the database to use
db_conn = nld.db.connect(filename=dbfilename)

# Then you can inspect the number of games in each dataset:
print(f"NLD AA \"Taster\" Dataset has {nld.db.count_games('taster-dataset', conn=db_conn)} games.")

NLD AA "Taster" Dataset has 1934 games.


## Visualizing the Data

Next, to actually load the games for training you'll use the `TtyrecDataset` object:

In [17]:
dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=32,
    seq_length=32,
    dbfilename=dbfilename,
)

This dataset above will return batches of 128 trajectories, returning sequential chunks of length 32.   That is, assuming the length of all trajectories is >>64, the first batch will give timesteps 0-31 of 128 games and the second batch will provide timesteps 32-63 for the same games, etc.

### Whats in the Observation?

In [18]:
minibatch = next(iter(dataset))
minibatch.keys()

dict_keys(['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores'])

The observation is made up of three components:
- `tty_chars` is a (batched) 2D np.array of the characters displayed at each point on the screen with shape: `[Batch, Time, H, W]`
- `tty_colors` is the associated colors for those characters
- `tty_cursor` provides the cursor position (NOTE: it's not always on the hero!)

These can be easily visualized usign the `tty_render` utility:

In [19]:
from nle.nethack import tty_render

In [20]:
batch_idx = 0
time_idx = 0
chars = minibatch['tty_chars'][batch_idx, time_idx]
colors = minibatch['tty_colors'][batch_idx, time_idx]
cursor = minibatch['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))


Hello Agent, welcome to NetHack!  You are a chaotic male orcish Barbarian.      
                                                                                
                                                        ----------              
                                                        .....@...|              
                                                        |.....d..|              
                                                        +......x)|              
                                                        ----------              
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                           

### Extracting Inventory Information from Dataset

The dataset contains several ways to access inventory information:
1. **From TTY rendering** - parsing the visual inventory display
2. **From minibatch keys** - if available in the dataset
3. **By triggering inventory commands** - looking for 'i' keypresses

Let's explore all these methods:

In [39]:
# First, let's see what keys are available in our minibatch
print("=== Available Data in Minibatch ===")
print("Keys:", list(minibatch.keys()))
print()

# Check if inventory-related keys exist
inventory_keys = ['inv_glyphs', 'inv_letters', 'inv_oclasses', 'inv_strs']
available_inv_keys = [key for key in inventory_keys if key in minibatch.keys()]
print(f"Inventory-specific keys available: {available_inv_keys}")

# If no direct inventory keys, we'll need to parse from TTY or find inventory screens
if not available_inv_keys:
    print("No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.")
else:
    print("Great! We have direct inventory data available.")

print(f"\nMinibatch shapes:")
for key, value in minibatch.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {type(value)}")

=== Available Data in Minibatch ===
Keys: ['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores']

Inventory-specific keys available: []
No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.

Minibatch shapes:
  tty_chars: (32, 32, 24, 80)
  tty_colors: (32, 32, 24, 80)
  tty_cursor: (32, 32, 2)
  timestamps: (32, 32)
  done: (32, 32)
  gameids: (32, 32)
  keypresses: (32, 32)
  scores: (32, 32)


In [40]:
# Method 1: Find when players opened inventory (pressed 'i')
print("=== Method 1: Finding Inventory Commands ===")

# Look for inventory keypresses (ASCII 105 = 'i')
inventory_keypress = ord('i')  # 105

# Search through the dataset for inventory commands
batch_with_inventory = None
timestep_with_inventory = None

for batch_idx in range(min(5, minibatch['keypresses'].shape[0])):  # Check first 5 batches
    for time_idx in range(minibatch['keypresses'].shape[1]):
        if minibatch['keypresses'][batch_idx, time_idx] == inventory_keypress:
            batch_with_inventory = batch_idx
            timestep_with_inventory = time_idx
            print(f"Found inventory command at batch {batch_idx}, timestep {time_idx}")
            break
    if batch_with_inventory is not None:
        break

if batch_with_inventory is None:
    print("No inventory commands found in current minibatch. Let's create a larger search...")
    
    # Create a larger dataset to search for inventory
    larger_dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=10,
        seq_length=100,
        dbfilename=dbfilename,
    )
    
    # Search multiple batches
    found_inventory = False
    for mb_idx, large_mb in enumerate(larger_dataset):
        if mb_idx > 3:  # Don't search forever
            break
            
        inventory_positions = (large_mb['keypresses'] == inventory_keypress)
        if inventory_positions.any():
            # Find the first occurrence
            batch_indices, time_indices = inventory_positions.nonzero()
            batch_with_inventory = batch_indices[0]
            timestep_with_inventory = time_indices[0]
            
            print(f"Found inventory command in batch {mb_idx}, game {batch_with_inventory}, timestep {timestep_with_inventory}")
            
            # Use this minibatch for analysis
            minibatch_with_inv = large_mb
            found_inventory = True
            break
    
    if not found_inventory:
        print("No inventory commands found. Using current minibatch for demonstration.")
        minibatch_with_inv = minibatch
        batch_with_inventory = 0
        timestep_with_inventory = 10  # Just pick a timestep
else:
    minibatch_with_inv = minibatch

print(f"Using batch {batch_with_inventory}, timestep {timestep_with_inventory} for analysis")

=== Method 1: Finding Inventory Commands ===
Found inventory command at batch 1, timestep 17
Using batch 1, timestep 17 for analysis


In [41]:
# Method 2: Analyze the inventory screen
print("=== Method 2: Analyzing Inventory Screen ===")

# Look at the screen right after the inventory command
if timestep_with_inventory + 1 < minibatch_with_inv['tty_chars'].shape[1]:
    next_timestep = timestep_with_inventory + 1
else:
    next_timestep = timestep_with_inventory

chars_before = minibatch_with_inv['tty_chars'][batch_with_inventory, timestep_with_inventory]
colors_before = minibatch_with_inv['tty_colors'][batch_with_inventory, timestep_with_inventory]
cursor_before = minibatch_with_inv['tty_cursor'][batch_with_inventory, timestep_with_inventory]

chars_after = minibatch_with_inv['tty_chars'][batch_with_inventory, next_timestep]
colors_after = minibatch_with_inv['tty_colors'][batch_with_inventory, next_timestep]
cursor_after = minibatch_with_inv['tty_cursor'][batch_with_inventory, next_timestep]

print("Screen BEFORE inventory command:")
print(tty_render(chars_before, colors_before, cursor_before))
print("\n" + "="*80 + "\n")

print("Screen AFTER inventory command:")
print(tty_render(chars_after, colors_after, cursor_after))

# Method 3: Extract inventory information from the text
print("\n=== Method 3: Parsing Inventory Information ===")

def extract_inventory_from_screen(chars):
    """Extract inventory items from TTY characters"""
    inventory_items = []
    
    # Convert chars to string representation
    screen_lines = []
    for row in range(chars.shape[0]):
        line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
        screen_lines.append(line.rstrip())
    
    # Look for common inventory patterns
    for i, line in enumerate(screen_lines):
        line = line.strip()
        
        # NetHack inventory lines typically start with a letter followed by ')'
        if len(line) > 2 and line[1] == ')' and line[0].isalpha():
            inventory_items.append({
                'slot': line[0],
                'description': line[2:].strip(),
                'line_number': i
            })
        
        # Also look for "You are carrying:" or similar inventory headers
        if 'carrying' in line.lower() or 'inventory' in line.lower():
            print(f"Inventory header found at line {i}: '{line}'")
    
    return inventory_items, screen_lines

# Extract from the inventory screen
inv_items, screen_lines = extract_inventory_from_screen(chars_after)

print(f"Found {len(inv_items)} inventory items:")
for item in inv_items:
    print(f"  {item['slot']}) {item['description']}")

if len(inv_items) == 0:
    print("No inventory items found in standard format.")
    print("Screen might show a different interface. Here are all non-empty lines:")
    for i, line in enumerate(screen_lines):
        if line.strip():
            print(f"  Line {i:2d}: '{line}'")

=== Method 2: Analyzing Inventory Screen ===
Screen BEFORE inventory command:

                                What do you want to name?                       
                                                                                
                                m - a monster                                   
                                i - a particular object in inventory            
                                o - the type of an object in inventory          
                                f - the type of an object upon the floor        
                                d - the type of an object on discoveries list   
                                a - record an annotation for the current level  
                                (end)                                           
                                                                                
                                                                                
                              

In [42]:
# Method 4: Advanced Inventory Analysis Tools
print("=== Method 4: Advanced Inventory Extraction ===")

def find_all_inventory_moments(dataset, max_batches=10):
    """Find all moments when inventory was accessed"""
    inventory_moments = []
    inventory_keypress = ord('i')
    
    for batch_idx, mb in enumerate(dataset):
        if batch_idx >= max_batches:
            break
            
        # Find all inventory keypresses
        inv_positions = (mb['keypresses'] == inventory_keypress)
        batch_indices, time_indices = inv_positions.nonzero()
        
        for b_idx, t_idx in zip(batch_indices, time_indices):
            inventory_moments.append({
                'batch_in_dataset': batch_idx,
                'game_in_batch': b_idx,
                'timestep': t_idx,
                'gameid': mb['gameids'][b_idx, t_idx],
                'minibatch': mb
            })
    
    return inventory_moments

def analyze_inventory_sequence(mb, batch_idx, start_time, window=5):
    """Analyze a sequence around inventory opening"""
    print(f"Analyzing inventory sequence for game {batch_idx}, starting at timestep {start_time}")
    
    results = []
    for offset in range(window):
        t_idx = start_time + offset
        if t_idx >= mb['tty_chars'].shape[1]:
            break
            
        chars = mb['tty_chars'][batch_idx, t_idx]
        colors = mb['tty_colors'][batch_idx, t_idx]
        cursor = mb['tty_cursor'][batch_idx, t_idx]
        keypress = mb['keypresses'][batch_idx, t_idx]
        
        # Extract inventory info
        inv_items, _ = extract_inventory_from_screen(chars)
        
        results.append({
            'timestep': t_idx,
            'offset_from_inventory': offset,
            'keypress': keypress,
            'keypress_char': chr(keypress) if 32 <= keypress <= 126 else f'\\x{keypress:02x}',
            'inventory_items': inv_items,
            'screen': tty_render(chars, colors, cursor)
        })
    
    return results

# Find inventory moments in current dataset
inv_moments = find_all_inventory_moments([minibatch_with_inv], max_batches=1)
print(f"Found {len(inv_moments)} inventory moments in current data")

if inv_moments:
    # Analyze the first inventory moment in detail
    moment = inv_moments[0]
    print(f"\\nAnalyzing inventory moment:")
    print(f"  Game ID: {moment['gameid']}")
    print(f"  Timestep: {moment['timestep']}")
    
    # Get sequence around inventory
    sequence = analyze_inventory_sequence(
        moment['minibatch'], 
        moment['game_in_batch'], 
        moment['timestep'], 
        window=3
    )
    
    print(f"\\nInventory sequence analysis:")
    for step in sequence:
        print(f"\\nTimestep {step['timestep']} (offset +{step['offset_from_inventory']}):")
        print(f"  Keypress: {step['keypress']} ('{step['keypress_char']}')")
        print(f"  Items found: {len(step['inventory_items'])}")
        for item in step['inventory_items'][:3]:  # Show first 3 items
            print(f"    {item['slot']}) {item['description'][:50]}...")
        
        if step['offset_from_inventory'] == 1:  # Show screen after inventory command
            print("  Screen after inventory command:")
            print("  " + "\\n  ".join(step['screen'].split('\\n')[:10]))  # First 10 lines

# Method 5: Create an inventory tracking utility
print("\\n=== Method 5: Inventory Tracking Utility ===")

class InventoryTracker:
    def __init__(self):
        self.inventory_cache = {}
    
    def extract_inventory_at_timestep(self, mb, batch_idx, time_idx):
        """Extract inventory at a specific timestep"""
        chars = mb['tty_chars'][batch_idx, time_idx]
        items, screen_lines = extract_inventory_from_screen(chars)
        
        # Additional parsing for different inventory formats
        parsed_items = []
        for item in items:
            parsed_item = self.parse_inventory_item(item['description'])
            parsed_item['slot'] = item['slot']
            parsed_items.append(parsed_item)
        
        return parsed_items
    
    def parse_inventory_item(self, description):
        """Parse item description to extract details"""
        item_info = {
            'full_description': description,
            'item_type': 'unknown',
            'is_equipped': False,
            'is_blessed': False,
            'is_cursed': False,
            'quantity': 1
        }
        
        desc_lower = description.lower()
        
        # Check for equipment status
        if '(being worn)' in desc_lower or '(wielded)' in desc_lower:
            item_info['is_equipped'] = True
        
        # Check for blessed/cursed status
        if 'blessed' in desc_lower:
            item_info['is_blessed'] = True
        elif 'cursed' in desc_lower:
            item_info['is_cursed'] = True
        
        # Try to extract quantity (for stackable items)
        import re
        quantity_match = re.search(r'^(\\d+)\\s+', description)
        if quantity_match:
            item_info['quantity'] = int(quantity_match.group(1))
        
        # Categorize item type based on keywords
        item_types = {
            'weapon': ['sword', 'dagger', 'bow', 'arrow', 'spear', 'mace'],
            'armor': ['armor', 'helm', 'shield', 'boots', 'gloves', 'cloak'],
            'food': ['food', 'ration', 'apple', 'orange', 'banana'],
            'potion': ['potion'],
            'scroll': ['scroll'],
            'wand': ['wand'],
            'ring': ['ring'],
            'tool': ['tool', 'key', 'lamp', 'pick-axe']
        }
        
        for category, keywords in item_types.items():
            if any(keyword in desc_lower for keyword in keywords):
                item_info['item_type'] = category
                break
        
        return item_info
    
    def track_inventory_changes(self, mb, batch_idx, start_time, end_time):
        """Track how inventory changes over time"""
        changes = []
        prev_items = None
        
        for t in range(start_time, min(end_time, mb['tty_chars'].shape[1])):
            current_items = self.extract_inventory_at_timestep(mb, batch_idx, t)
            
            if prev_items is not None and current_items != prev_items:
                changes.append({
                    'timestep': t,
                    'added': [item for item in current_items if item not in prev_items],
                    'removed': [item for item in prev_items if item not in current_items]
                })
            
            prev_items = current_items
        
        return changes

# Example usage
tracker = InventoryTracker()
if inv_moments:
    moment = inv_moments[0]
    items = tracker.extract_inventory_at_timestep(
        moment['minibatch'], 
        moment['game_in_batch'], 
        moment['timestep'] + 1  # Look at screen after inventory command
    )
    
    print(f"Detailed inventory analysis:")
    for item in items[:5]:  # Show first 5 items
        print(f"  Slot {item['slot']}: {item['item_type']} - {item['full_description'][:40]}...")
        if item['is_equipped']:
            print(f"    → Currently equipped")
        if item['quantity'] > 1:
            print(f"    → Quantity: {item['quantity']}")

print(f"\\n🎯 Summary: You can extract inventory information by:")
print(f"1. Finding 'i' keypresses (inventory commands)")
print(f"2. Parsing the TTY screen content after inventory commands")
print(f"3. Using regex patterns to extract item details")
print(f"4. Tracking inventory changes over time")
print(f"5. Building custom parsers for different inventory screens")

=== Method 4: Advanced Inventory Extraction ===
Found 3 inventory moments in current data
\nAnalyzing inventory moment:
  Game ID: 285
  Timestep: 17
Analyzing inventory sequence for game 1, starting at timestep 17
Inventory header found at line 3: 'i - a particular object in inventory'
Inventory header found at line 4: 'o - the type of an object in inventory'
\nInventory sequence analysis:
\nTimestep 17 (offset +0):
  Keypress: 105 ('i')
  Items found: 0
\nTimestep 18 (offset +1):
  Keypress: 104 ('h')
  Items found: 0
  Screen after inventory command:
  
What do you want to name? [a-i or ?*]                                           
                                                                                
                                                        ----------              
                                                        |.......@|              
                                                        |......xf|              
                                

In [43]:
# Practical Usage for RL Research
print("=== Practical Usage for Sequential Skill RL Research ===")

def create_inventory_features(mb, batch_idx, time_idx):
    """Create feature vector from inventory state"""
    # Extract inventory
    chars = mb['tty_chars'][batch_idx, time_idx]
    items, _ = extract_inventory_from_screen(chars)
    
    # Create feature vector
    features = {
        'total_items': len(items),
        'equipped_items': 0,
        'food_items': 0,
        'weapon_items': 0,
        'armor_items': 0,
        'potion_items': 0,
        'tool_items': 0,
        'carrying_capacity_used': len(items) / 52.0  # NetHack max ~52 items
    }
    
    # Count by category
    for item in items:
        desc_lower = item['description'].lower()
        
        if 'being worn' in desc_lower or 'wielded' in desc_lower:
            features['equipped_items'] += 1
        
        if any(food in desc_lower for food in ['food', 'ration', 'apple', 'orange']):
            features['food_items'] += 1
        elif any(weapon in desc_lower for weapon in ['sword', 'dagger', 'bow', 'arrow']):
            features['weapon_items'] += 1
        elif any(armor in desc_lower for armor in ['armor', 'helm', 'shield', 'boots']):
            features['armor_items'] += 1
        elif 'potion' in desc_lower:
            features['potion_items'] += 1
        elif any(tool in desc_lower for tool in ['tool', 'key', 'lamp', 'pick-axe']):
            features['tool_items'] += 1
    
    return features

def detect_inventory_management_skills(mb, batch_idx, window_size=10):
    """Detect inventory management patterns (potential skills)"""
    skills_detected = []
    
    for t in range(window_size, mb['keypresses'].shape[1] - window_size):
        keypress_window = mb['keypresses'][batch_idx, t-window_size:t+window_size]
        
        # Convert to characters
        chars = [chr(k) if 32 <= k <= 126 else None for k in keypress_window]
        
        # Detect patterns
        patterns = {
            'inventory_management': ['i', 'd', 'i'],  # open inventory, drop, check inventory
            'equipment_change': ['i', 'w', 'a'],      # inventory, wield, select item
            'consumption': ['i', 'e', 'a'],           # inventory, eat, select food
            'potion_use': ['i', 'q', 'a'],            # inventory, quaff, select potion
            'item_identification': ['i', '/', 'a']     # inventory, whatis, select item
        }
        
        # Check for pattern matches
        for skill_name, pattern in patterns.items():
            if len(pattern) <= len(chars):
                for start_idx in range(len(chars) - len(pattern) + 1):
                    match = True
                    for i, p in enumerate(pattern):
                        if chars[start_idx + i] != p:
                            match = False
                            break
                    
                    if match:
                        skills_detected.append({
                            'skill': skill_name,
                            'timestep': t - window_size + start_idx,
                            'pattern': pattern,
                            'confidence': 1.0  # Could implement more sophisticated scoring
                        })
    
    return skills_detected

# Example usage with current data
print("Creating inventory features for sample timesteps:")

sample_timesteps = [0, 5, 10, 15] if minibatch_with_inv['tty_chars'].shape[1] > 15 else [0, 1, 2]

for t in sample_timesteps:
    features = create_inventory_features(minibatch_with_inv, batch_with_inventory, t)
    print(f"\\nTimestep {t}:")
    for feature, value in features.items():
        print(f"  {feature}: {value}")

# Detect inventory management skills
print(f"\\nDetecting inventory management skills:")
skills = detect_inventory_management_skills(minibatch_with_inv, batch_with_inventory)

if skills:
    print(f"Found {len(skills)} potential inventory management skills:")
    for skill in skills[:5]:  # Show first 5
        print(f"  {skill['skill']} at timestep {skill['timestep']} (pattern: {skill['pattern']})")
else:
    print("No clear inventory management patterns detected in this sequence.")

print(f"\\n🚀 Applications for Sequential Skill RL:")
print(f"1. **State Augmentation**: Add inventory features to observation space")
print(f"2. **Skill Discovery**: Identify inventory management as discrete skills")
print(f"3. **Reward Shaping**: Reward efficient inventory usage")
print(f"4. **Hierarchical RL**: Use inventory state for high-level planning")
print(f"5. **Transfer Learning**: Learn inventory skills that transfer across games")
print(f"6. **Behavioral Cloning**: Clone human inventory management strategies")

print(f"\\n📊 Example Feature Engineering:")
print(f"- Inventory diversity (Shannon entropy of item types)")
print(f"- Equipment optimization (% of equipment slots filled)")
print(f"- Resource management (food/potion ratios)")
print(f"- Inventory turnover (items picked up vs dropped)")
print(f"- Strategic depth (tools for specific dungeon levels)")

=== Practical Usage for Sequential Skill RL Research ===
Creating inventory features for sample timesteps:
\nTimestep 0:
  total_items: 0
  equipped_items: 0
  food_items: 0
  weapon_items: 0
  armor_items: 0
  potion_items: 0
  tool_items: 0
  carrying_capacity_used: 0.0
\nTimestep 5:
  total_items: 0
  equipped_items: 0
  food_items: 0
  weapon_items: 0
  armor_items: 0
  potion_items: 0
  tool_items: 0
  carrying_capacity_used: 0.0
\nTimestep 10:
  total_items: 0
  equipped_items: 0
  food_items: 0
  weapon_items: 0
  armor_items: 0
  potion_items: 0
  tool_items: 0
  carrying_capacity_used: 0.0
\nTimestep 15:
  total_items: 0
  equipped_items: 0
  food_items: 0
  weapon_items: 0
  armor_items: 0
  potion_items: 0
  tool_items: 0
  carrying_capacity_used: 0.0
\nDetecting inventory management skills:
No clear inventory management patterns detected in this sequence.
\n🚀 Applications for Sequential Skill RL:
1. **State Augmentation**: Add inventory features to observation space
2. **Sk

### Understanding NetHack Interface Behavior

You've made excellent observations! Let me explain what's happening with the NetHack interface:

#### Why "What do you want to name?" appears before pressing 'i':
1. **NetHack Interface State**: NetHack remembers previous interactions and menu states
2. **Menu Persistence**: Some menus/prompts can persist across multiple keypresses
3. **Context-Dependent Actions**: The 'i' key might be responding to an existing prompt, not opening inventory

#### Why inventory doesn't show in `a)` format:
1. **Empty Inventory**: Player might have no items (common early in game)
2. **Different Display Mode**: NetHack has multiple inventory display modes
3. **Menu State**: The 'i' might be triggering a different action due to current context
4. **Interface Variations**: Different NetHack versions/configurations show inventory differently

Let's investigate these patterns more carefully:

In [44]:
# Let's investigate the confusing interface behavior you observed
print("=== Investigating NetHack Interface Behavior ===")

# Let's look more carefully at the sequence around the 'i' keypress
def analyze_interface_context(mb, batch_idx, timestep, window=5):
    """Analyze the context around a keypress to understand interface state"""
    print(f"Analyzing interface context around timestep {timestep}")
    
    start_time = max(0, timestep - window)
    end_time = min(mb['tty_chars'].shape[1], timestep + window + 1)
    
    for t in range(start_time, end_time):
        keypress = mb['keypresses'][batch_idx, t]
        chars = mb['tty_chars'][batch_idx, t]
        
        # Convert screen to text
        screen_lines = []
        for row in range(chars.shape[0]):
            line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
            screen_lines.append(line.rstrip())
        
        # Look for prompts and menus
        prompts_found = []
        inventory_items = []
        
        for i, line in enumerate(screen_lines):
            line_lower = line.lower().strip()
            
            # Look for various NetHack prompts
            if any(prompt in line_lower for prompt in [
                'what do you want', 'name', 'call', 'select', 'choose',
                'which', 'how many', 'direction', 'really', 'are you sure'
            ]):
                prompts_found.append(f"Line {i}: '{line.strip()}'")
            
            # Look for inventory format
            if len(line.strip()) > 2 and line.strip()[1] == ')' and line.strip()[0].isalpha():
                inventory_items.append(f"Line {i}: '{line.strip()}'")
        
        # Character representation of keypress
        char_repr = chr(keypress) if 32 <= keypress <= 126 else f'\\x{keypress:02x}'
        
        print(f"\\nTimestep {t} (offset {t-timestep:+d}):")
        print(f"  Keypress: {keypress} ('{char_repr}')")
        print(f"  Prompts found: {len(prompts_found)}")
        for prompt in prompts_found[:2]:  # Show first 2 prompts
            print(f"    {prompt}")
        print(f"  Inventory items found: {len(inventory_items)}")
        for item in inventory_items[:2]:  # Show first 2 items
            print(f"    {item}")
        
        # Check for specific screens
        screen_text = ' '.join(screen_lines).lower()
        screen_type = "unknown"
        
        if 'inventory' in screen_text or 'carrying' in screen_text:
            screen_type = "inventory_screen"
        elif 'what do you want' in screen_text:
            screen_type = "prompt_screen"
        elif 'dungeon' in screen_text or len([l for l in screen_lines if '#' in l or '.' in l]) > 5:
            screen_type = "game_map"
        elif any(word in screen_text for word in ['menu', 'select', 'choose']):
            screen_type = "menu_screen"
        
        print(f"  Screen type: {screen_type}")
        
        if t == timestep:  # The actual 'i' keypress
            print(f"  *** THIS IS THE 'i' KEYPRESS ***")
            print(f"  Full screen preview:")
            for i, line in enumerate(screen_lines[:10]):  # First 10 lines
                if line.strip():
                    print(f"    {i:2d}: '{line}'")

# Analyze the inventory keypress we found earlier
if 'batch_with_inventory' in locals() and 'timestep_with_inventory' in locals():
    analyze_interface_context(minibatch_with_inv, batch_with_inventory, timestep_with_inventory, window=3)
else:
    print("No inventory keypress found to analyze")

print("\\n=== Key Insights About NetHack Interface ===")
print("1. **Context Matters**: The 'i' key doesn't always open inventory")
print("2. **State Persistence**: Prompts and menus can carry over between actions")
print("3. **Empty Inventory**: Early game characters often have no items")
print("4. **Multiple Meanings**: Same key can do different things in different contexts")
print("5. **Interface Variations**: NetHack UI can look different based on game state")

print("\\n=== Common 'i' Key Behaviors ===")
print("- **Normal gameplay**: Opens inventory")
print("- **During prompt**: Might select an option or answer a question")
print("- **In menu**: Might navigate or select items")
print("- **Naming context**: Might be part of typing a name")
print("- **Empty inventory**: Shows 'You are not carrying anything'")

print("\\n=== Better Inventory Detection Strategy ===")
print("1. Look for 'You are carrying:' or 'You are not carrying anything'")
print("2. Check for screen changes after 'i' keypress")
print("3. Parse different inventory display formats")
print("4. Consider game context and state")

=== Investigating NetHack Interface Behavior ===
Analyzing interface context around timestep 17
\nTimestep 14 (offset -3):
  Keypress: 32 (' ')
  Prompts found: 0
  Inventory items found: 0
  Screen type: game_map
\nTimestep 15 (offset -2):
  Keypress: 32 (' ')
  Prompts found: 0
  Inventory items found: 0
  Screen type: unknown
\nTimestep 16 (offset -1):
  Keypress: 67 ('C')
  Prompts found: 0
  Inventory items found: 0
  Screen type: unknown
\nTimestep 17 (offset +0):
  Keypress: 105 ('i')
  Prompts found: 1
    Line 0: 'What do you want to name?'
  Inventory items found: 0
  Screen type: inventory_screen
  *** THIS IS THE 'i' KEYPRESS ***
  Full screen preview:
     0: '                                What do you want to name?'
     2: '                                m - a monster'
     3: '                                i - a particular object in inventory'
     4: '                                o - the type of an object in inventory'
     5: '                                f 

In [46]:
# More robust inventory detection system
print("=== Improved Inventory Detection ===")

def detect_inventory_screens(mb, batch_idx, max_timesteps=None):
    """Detect actual inventory screens, not just 'i' keypresses"""
    if max_timesteps is None:
        max_timesteps = mb['tty_chars'].shape[1]
    
    # Ensure we don't exceed available timesteps
    max_timesteps = min(max_timesteps, mb['tty_chars'].shape[1])
    
    inventory_screens = []
    
    for t in range(max_timesteps):
        chars = mb['tty_chars'][batch_idx, t]
        
        # Convert to text
        screen_lines = []
        for row in range(chars.shape[0]):
            line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
            screen_lines.append(line.rstrip())
        
        screen_text = ' '.join(screen_lines).lower()
        
        # Multiple ways to detect inventory screens
        inventory_indicators = [
            'you are carrying:',
            'you are not carrying anything',
            'inventory:',
            'your possessions:',
            'items carried:',
        ]
        
        # Check for inventory indicators
        is_inventory = any(indicator in screen_text for indicator in inventory_indicators)
        
        # Also check for the a) b) c) pattern (if items exist)
        item_pattern_count = 0
        for line in screen_lines:
            line = line.strip()
            if len(line) > 2 and line[1] == ')' and line[0].isalpha():
                item_pattern_count += 1
        
        # If we have multiple a) b) c) patterns, it's likely inventory
        if item_pattern_count >= 2:
            is_inventory = True
        
        if is_inventory:
            # Extract items
            items = []
            for i, line in enumerate(screen_lines):
                line = line.strip()
                if len(line) > 2 and line[1] == ')' and line[0].isalpha():
                    items.append({
                        'slot': line[0],
                        'description': line[2:].strip(),
                        'line_number': i
                    })
            
            inventory_screens.append({
                'timestep': t,
                'items': items,
                'screen_lines': screen_lines,
                'item_count': len(items),
                'is_empty': 'not carrying anything' in screen_text
            })
    
    return inventory_screens

def find_actual_inventory_moments(dataset, max_batches=5):
    """Find moments with actual inventory screens"""
    all_inventory_moments = []
    
    for batch_idx, mb in enumerate(dataset):
        if batch_idx >= max_batches:
            break
        
        for game_idx in range(mb['tty_chars'].shape[0]):
            # Use the actual sequence length for this game
            max_timesteps = mb['tty_chars'].shape[1]
            inventory_screens = detect_inventory_screens(mb, game_idx, max_timesteps=max_timesteps)
            
            for screen in inventory_screens:
                all_inventory_moments.append({
                    'batch_in_dataset': batch_idx,
                    'game_in_batch': game_idx,
                    'timestep': screen['timestep'],
                    'gameid': mb['gameids'][game_idx, screen['timestep']],
                    'items': screen['items'],
                    'item_count': screen['item_count'],
                    'is_empty': screen['is_empty'],
                    'minibatch': mb
                })
    
    return all_inventory_moments

# Search for actual inventory screens
print("Searching for actual inventory screens...")
print(f"Dataset shape: {minibatch_with_inv['tty_chars'].shape}")

real_inventory_moments = find_actual_inventory_moments([minibatch_with_inv], max_batches=1)

print(f"Found {len(real_inventory_moments)} actual inventory screens")

if real_inventory_moments:
    for i, moment in enumerate(real_inventory_moments[:3]):  # Show first 3
        print(f"\\nInventory Screen {i+1}:")
        print(f"  Timestep: {moment['timestep']}")
        print(f"  Game ID: {moment['gameid']}")
        print(f"  Items: {moment['item_count']}")
        print(f"  Empty: {moment['is_empty']}")
        
        if moment['items']:
            print(f"  Items found:")
            for item in moment['items'][:3]:
                print(f"    {item['slot']}) {item['description']}")
        else:
            print(f"  No items (empty inventory or different format)")

else:
    print("No clear inventory screens found.")
    print("This suggests:")
    print("1. Players might not have opened inventory in this dataset segment")
    print("2. All inventories were empty")
    print("3. The interface is showing different formats than expected")

# Let's also search for "what do you want to name" contexts
print("\\n=== Investigating 'Name' Prompts ===")

def find_naming_contexts(mb, batch_idx, max_timesteps=None):
    """Find 'what do you want to name' contexts"""
    if max_timesteps is None:
        max_timesteps = mb['tty_chars'].shape[1]
    
    max_timesteps = min(max_timesteps, mb['tty_chars'].shape[1])
    naming_contexts = []
    
    for t in range(max_timesteps):
        chars = mb['tty_chars'][batch_idx, t]
        
        # Convert to text
        screen_lines = []
        for row in range(chars.shape[0]):
            line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
            screen_lines.append(line.rstrip())
        
        screen_text = ' '.join(screen_lines).lower()
        
        if 'what do you want to name' in screen_text or 'call' in screen_text:
            # Find the specific keypress that led to this
            keypress = mb['keypresses'][batch_idx, t-1] if t > 0 else 0
            char_repr = chr(keypress) if 32 <= keypress <= 126 else f'\\x{keypress:02x}'
            
            naming_contexts.append({
                'timestep': t,
                'previous_keypress': keypress,
                'previous_char': char_repr,
                'screen_lines': screen_lines
            })
    
    return naming_contexts

naming_moments = find_naming_contexts(minibatch_with_inv, batch_with_inventory)

if naming_moments:
    print(f"Found {len(naming_moments)} naming prompts:")
    for moment in naming_moments[:2]:
        print(f"  Timestep {moment['timestep']}: Previous key was {moment['previous_keypress']} ('{moment['previous_char']}')")
        relevant_lines = [line for line in moment['screen_lines'] if 'name' in line.lower() or 'call' in line.lower()]
        for line in relevant_lines[:2]:
            print(f"    '{line}'")
else:
    print("No naming prompts found")

print("\\n🔍 **Your Confusion Explained:**")
print("1. **'What do you want to name?' before 'i'**: NetHack's interface state was already in a naming mode")
print("2. **No a) format inventory**: The inventory was likely empty or in a different display mode")
print("3. **Context-dependent keys**: The 'i' key does different things based on current game state")
print("4. **Interface persistence**: NetHack menus and prompts can persist across multiple keypresses")

print("\\n💡 **Better Strategy for Dataset Analysis:**")
print("1. Look for actual inventory content, not just 'i' keypresses")
print("2. Parse different inventory formats (empty, full, menu-style)")
print("3. Consider the full interface context, not isolated actions")
print("4. Account for NetHack's stateful interface behavior")

=== Improved Inventory Detection ===
Searching for actual inventory screens...
Dataset shape: (32, 32, 24, 80)
Found 0 actual inventory screens
No clear inventory screens found.
This suggests:
1. Players might not have opened inventory in this dataset segment
2. All inventories were empty
3. The interface is showing different formats than expected
\n=== Investigating 'Name' Prompts ===
Found 6 naming prompts:
  Timestep 17: Previous key was 67 ('C')
    '                                What do you want to name?'
  Timestep 18: Previous key was 105 ('i')
    'What do you want to name? [a-i or ?*]'
\n🔍 **Your Confusion Explained:**
1. **'What do you want to name?' before 'i'**: NetHack's interface state was already in a naming mode
2. **No a) format inventory**: The inventory was likely empty or in a different display mode
3. **Context-dependent keys**: The 'i' key does different things based on current game state
4. **Interface persistence**: NetHack menus and prompts can persist across

Then, the other elements of the batch are:
- `gameids`: The gameid for the game which the observation is from.
- `timestamps`: The time when the state was recorded, allowing you to understand how long the player took between frames.
- `keypresses`: The keypresses entered after seeing the observation at this timestep (which produces the observation at the next timestep).
- `scores`: The in-game score at this timestep (the result of the action at the previous timestep)
- `done`: Whether the gameid corresponding to the previous timestep's observation completed. If done is `True` this means that the observation at the current timestep is the beginning of the next gameid.

### Converting Actions from Keypresses to Environment Action Space

Note that the "actions" data is actually a keypress (eg ascii) entered not an action value corresponding to the actions in the nle environment.  To convert from keypresses to the action_space of the environment you can use an embedding as shown below:

In [35]:
import torch
from nle.env.tasks import NetHackChallenge


env = NetHackChallenge(
    savedir=None,  # Do not save any recordings. 
    character='@', # Randomly rotate through characters.
)

# Then use the environment actions to convert the keypresses.
embed_actions = torch.zeros((256, 1))
for i, a in enumerate(env.actions):
    embed_actions[a.value][0] = i
    
embed_actions = torch.nn.Embedding.from_pretrained(embed_actions)
keypresses = torch.Tensor(minibatch["keypresses"]).long()
actions = embed_actions(keypresses).squeeze(-1).long()

## Dataset Configuration Options
`shuffle`: While states within a trajectory are always returned sequentially, it is possible to turn on shuffling of the *gameids*.  When true, the order of the gameids sampled is shuffled but not the order of the `seq_length` chunks returned within a single gameid.

`loop_forever`: It is possible to have the iterator loop forever instead of cycling only through the dataset once.

`gameids`: You can specify a list of gameids to return instead of iterating through the full dataset.

`subselect_sql`: And, you can select even more complicated sets of games using specific sql queries.

**NB** A `gameid` of 0 indicates that that index is padded (with 0's).

**Example 1:** Lets create a small dataset of just 4 games, and see the shuffle functionality:

In [26]:
shuffle_small_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=2,
    seq_length=6000,
    dbfilename=dbfilename,
    shuffle=True,
    loop_forever=False,
    gameids=[34,550,45],
)
for epoch in range(3):
    print(f"Epoch: {epoch}")
    for ind, mb in enumerate(shuffle_small_dataset):
        gameids = mb["gameids"][:, 0]
        print(f"  Batch {ind} first timestep gameids: {gameids}")
    print()


Epoch: 0
  Batch 0 first timestep gameids: [550  34]
  Batch 0 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 3 first timestep gameids: [550  45]
  Batch 3 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
 

**Example 2:** We can train just on the data from a specific character, such as "mon-hum-neu-mal" by using the subselect_sql:

In [32]:
# Build the subselect sql query
subselect_sql = "SELECT gameid FROM games WHERE role=? AND race=?"
subselect_sql_args = ("Mon", "Hum")
batch_size = 10

# Build the dataset
monk_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=batch_size,
    seq_length=2,
    dbfilename=dbfilename,
    subselect_sql=subselect_sql,
    subselect_sql_args=subselect_sql_args
)

# See from the error how there are fewer than 10k games despite the full dataset having 109k
print(f"Full Dataset has {nld.db.count_games('taster-dataset', conn=db_conn):,} games.")
print(f"Human Monk Subdataset Has: {len(monk_dataset._gameids)} games")

mb = next(iter(monk_dataset))

batch_idx = 0
time_idx = 0
chars = mb['tty_chars'][batch_idx, time_idx]
colors = mb['tty_colors'][batch_idx, time_idx]
cursor = mb['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))

Full Dataset has 1,934 games.
Human Monk Subdataset Has: 142 games

Hello Agent, welcome to NetHack!  You are a neutral female human Monk.          
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                         

**Example 3**: Using a threadpool
You can also use a threadpool with the dataset which will speed it up considerably!

In [34]:
from concurrent.futures import ThreadPoolExecutor
import time


with ThreadPoolExecutor(max_workers=10) as tp:
    dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=100,
        seq_length=100,
        dbfilename=dbfilename,
        threadpool=tp
    )
    start = time.time()
    for i, mb in enumerate(dataset):
        if i == 10:
            break
    end = time.time()
    chars = mb['tty_chars'][batch_idx, time_idx]
    colors = mb['tty_colors'][batch_idx, time_idx]
    cursor = mb['tty_cursor'][batch_idx, time_idx]

    print(tty_render(chars, colors, cursor))
# NB this might be v slow on free Colab, try on laptop or server.
print(f"Loaded 100,000 frames in {end-start:.2f}s")


                                                                                
                                                                                
                              -----------                                       
                              |..Z......|              ----------------         
                              |..........#           ##...:...........|         
                              |.........|#           # |._............|         
                              ---------|-#           # |..............|         
        ----------------            #### #           # -.--------------         
        |.......{..<...-###    ######    ##          #  #####                   
        |..............|  ##   #          #        ###      #                   
        |...............@#d#####          #      ###        #######             
        ----------------    #-|-----      #-----##                #             
                           

**Example 4:** Getting Metadata

In [ ]:
dataset = nld.TtyrecDataset('taster-dataset', dbfilename=dbfilename)
mb = next(iter(dataset))
gameid = mb["gameids"][0][0]

chars = mb['tty_chars'][0, 0]
colors = mb['tty_colors'][0, 0]
cursor = mb['tty_cursor'][0, 0]

print(tty_render(chars, colors, cursor))

dict(dataset.get_meta(gameid))


Hello Agent, welcome to NetHack!  You are a neutral male gnomish Archeologist.  
                                                                                
                                                                                
                                                                                
                               --------                                         
                               |....$.|                                         
                               |..!{..|                                         
                               |......|                                         
                               |...@f(|                                         
                               |......|                                         
                               |.....)|                                         
                               --.-----                                         
                           

{'gameid': 816,
 'version': '3.6.6',
 'points': 7515,
 'deathdnum': 0,
 'deathlev': 5,
 'maxlvl': 5,
 'hp': 0,
 'maxhp': 49,
 'deaths': 1,
 'deathdate': 20220518,
 'birthdate': 20220518,
 'uid': 1185200751,
 'role': 'Arc',
 'race': 'Gno',
 'gender': 'Mal',
 'align': 'Neu',
 'name': 'Agent',
 'death': 'killed by a white unicorn',
 'conduct': '0xfc0',
 'turns': 22750,
 'achieve': '0x0',
 'realtime': 142,
 'starttime': 1652882603,
 'endtime': 1652882745,
 'gender0': 'Mal',
 'align0': 'Neu',
 'flags': '0x4'}

**Example 5** Generating and loading a custom dataset.

In [ ]:
import gym
import nle
import nle.dataset as nld
from datetime import datetime

def generate_rollouts(env):
    obs = env.reset()
    episodes = 0
    while episodes < 10:
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            env.reset()
            episodes += 1

# 1. Create some envs, with a savedir directory 'path/to/save/X'
envA = gym.make("NetHackChallenge-v0", savedir="path/to/save/A", save_ttyrec_every=2)
envB = gym.make("NetHackScore-v0", character="Mon-Hum-Neu-Mal", savedir="path/to/save/B", save_ttyrec_every=1)

# 2. Generate rollouts
generate_rollouts(envA)
generate_rollouts(envB)

# 3. Add to directory, with given unique dataset name
name = f"dataset_{datetime.now().time()}"
if not nld.db.exists():
    nld.db.create()
nld.add_nledata_directory("path/to/save", name)

# 4. Use and enjoy!
dataset = nld.TtyrecDataset(name)
print(f"Dataset has {len(dataset._gameids)} entries!")



Adding dataset 'dataset_15:38:53.943302' ('path/to/save') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.00 sec. Size: 0.65 MB, Games: 15
Dataset has 15 entries!


**Example 6:** Use doctstrings - don't forget a lot of the classes and methods have docstrings. Have fun!

In [ ]:
help(nld.TtyrecDataset)

Help on class TtyrecDataset in module nle.dataset.dataset:

class TtyrecDataset(builtins.object)
 |  TtyrecDataset(dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |  
 |  Dataset object to allow iteration through the ttyrecs found in our ttyrec
 |  database.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |      An iterable dataset to load minibatches of NetHack games from compressed
 |      ttyrec*.bz2 files into numpy arrays. (shape: [batch_size, seq_length, ...])
 |      
 |      This class makes use of a sqlite3 database at `dbfilename` to find the
 |      metadata and the location of files in a dataset. It then uses these to
 |   